In [1]:
!pip install merlin-core

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 7.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 957.1/957.1 kB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.

In [2]:
!pip install nvtabular

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for merlin-dataloader: filename=merlin_dataloader-23.2.1-py3-none-any.whl size=40725 sha256=10c0cf15bc3297235ad062ef9173b3eeb8e0956a03a365f04049d4c5a528a15f
  Stored in directory: /root/.cache/pip/wheels/34/bb/93/d35bf8e031723b123fc587e5b201e769d6398f30f2d746f17c
Successfully built merlin-dataloader


In [3]:
!pip install merlin-models

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 392.1/392.1 kB 10.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for merlin-models: filename=merlin_models-23.2.0-py3-none-any.whl size=371141 sha256=d2c3f18547d27deaa9c2cbc82c1d83fbe5f76c06e6a2cfbac34f39d97d25f3ae
  Stored in directory: /root/.cache/pip/wheels/af/60/7f/5dd12f4c95d06237dc60140170042f2cac4972320b2cbdda5a
Successfully built merlin-models


In [4]:
!pip install redis==4.4.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.8/237.8 kB 7.8 MB/s eta 0:00:00


### 1 - Importing Libraries

In [5]:
import os
import logging
import time
import warnings
warnings.filterwarnings('ignore')

import nvtabular as nvt
import merlin.models.tf as mm
import tensorflow as tf

from nvtabular.ops import *

from merlin.datasets.synthetic import generate_data
from merlin.datasets.ecommerce import transform_aliccp
from merlin.models.utils.example_utils import workflow_fit_transform
from merlin.models.utils.dataset import unique_rows_by_features
from merlin.schema.tags import Tags
from merlin.io.dataset import Dataset


# disable INFO and DEBUG logging everywhere
logging.disable(logging.WARNING)

### 2 - Generate and Process Synthetic Ali-CCP Dataset

In [6]:
# Generate the synthetic data
NUM_ROWS = 1000000
TRAIN_SIZE = 0.7
VALID_SIZE = 0.3

train, valid = generate_data("aliccp-raw", NUM_ROWS, set_sizes=(TRAIN_SIZE, VALID_SIZE))

In [7]:
# Define output path for data
DATA_DIR = "/model-data/aliccp"
OUTPUT_DATA_DIR = os.path.join(DATA_DIR, "processed")
OUTPUT_RETRIEVAL_DATA_DIR = os.path.join(OUTPUT_DATA_DIR, "retrieval")
CATEGORY_TEMP_DIR = os.path.join(DATA_DIR, "categories")

In [8]:
# Define NVTabular Feature Transformation Pipeline

user_id_raw = ["user_id"] >> Rename(postfix='_raw') >> LambdaOp(lambda col: col.astype("int32")) >> TagAsUserFeatures()
item_id_raw = ["item_id"] >> Rename(postfix='_raw') >> LambdaOp(lambda col: col.astype("int32")) >> TagAsItemFeatures()

user_id = ["user_id"] >> Categorify(dtype="int32", out_path=CATEGORY_TEMP_DIR) >> TagAsUserID()
item_id = ["item_id"] >> Categorify(dtype="int32", out_path=CATEGORY_TEMP_DIR) >> TagAsItemID()

item_features = (
    ["item_category", "item_shop", "item_brand"] >> Categorify(dtype="int32", out_path=CATEGORY_TEMP_DIR) >> TagAsItemFeatures()
)

user_features = (
    [
        "user_shops",
        "user_profile",
        "user_group",
        "user_gender",
        "user_age",
        "user_consumption_2",
        "user_is_occupied",
        "user_geography",
        "user_intentions",
        "user_brands",
        "user_categories",
    ]
    >> Categorify(dtype="int32", out_path=CATEGORY_TEMP_DIR)
    >> TagAsUserFeatures()
)

targets = ["click"] >> AddMetadata(tags=[Tags.BINARY_CLASSIFICATION, "target"])

outputs = user_id + item_id + item_features + user_features +  user_id_raw + item_id_raw + targets

# add dropna op to filter rows with nulls
outputs = outputs >> Dropna()

In [9]:
# Transform data and create files
transform_aliccp((train, valid), OUTPUT_DATA_DIR, nvt_workflow=outputs)

### 3 - Candidate Retrieval Model

#### 3.1 - Feature Engineering for Candidate Retrieval

In [10]:
# Load Datasets from generated files
train_retrieval = Dataset(os.path.join(OUTPUT_DATA_DIR, "train", "*.parquet"))
valid_retrieval = Dataset(os.path.join(OUTPUT_DATA_DIR, "valid", "*.parquet"))

In [11]:
# Define NVTabular Feature Transformation Pipeline

inputs = train_retrieval.schema.column_names

# Select only positive interaction rows where click==1 in the dataset with Filter() operator
outputs = inputs >> Filter(f=lambda df: df["click"] == 1)

# Execute the transformation workflow for both datasets
workflow = nvt.Workflow(outputs)
workflow.fit(train_retrieval)
workflow.transform(train_retrieval).to_parquet(
    output_path=os.path.join(OUTPUT_RETRIEVAL_DATA_DIR, "train")
)
workflow.transform(valid_retrieval).to_parquet(
    output_path=os.path.join(OUTPUT_RETRIEVAL_DATA_DIR, "valid")
)

In [12]:
# Read transformed parquet files as Dataset objects
train_retrieval = Dataset(os.path.join(OUTPUT_RETRIEVAL_DATA_DIR, "train", "*.parquet"), part_size="500MB")
valid_retrieval = Dataset(os.path.join(OUTPUT_RETRIEVAL_DATA_DIR, "valid", "*.parquet"), part_size="500MB")

In [13]:
# Create model input schema
schema = train_retrieval.schema.select_by_tag([Tags.ITEM_ID, Tags.USER_ID, Tags.ITEM, Tags.USER]).without(['user_id_raw', 'item_id_raw', 'click'])
train_retrieval.schema = schema
valid_retrieval.schema = schema

In [ ]:
schema

,name,tags,dtype,is_list,is_ragged,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.num_buckets,properties.freq_threshold,properties.start_index,properties.max_size,properties.cat_path,properties.domain.min,properties.domain.max,properties.domain.name
0,user_id,"(Tags.CATEGORICAL, Tags.ID, Tags.USER_ID, Tags...","DType(name='int32', element_type=<ElementType....",False,False,780.0,67.0,None,0.0,0.0,0.0,/model-data/aliccp/categories/categories/uniqu...,0,779,user_id
1,item_id,"(Tags.ITEM_ID, Tags.CATEGORICAL, Tags.ID, Tags...","DType(name='int32', element_type=<ElementType....",False,False,756.0,65.0,None,0.0,0.0,0.0,/model-data/aliccp/categories/categories/uniqu...,0,755,item_id
2,item_category,"(Tags.ITEM, Tags.CATEGORICAL)","DType(name='int32', element_type=<ElementType....",False,False,756.0,65.0,None,0.0,0.0,0.0,/model-data/aliccp/categories/categories/uniqu...,0,755,item_category
3,item_shop,"(Tags.ITEM, Tags.CATEGORICAL)","DType(name='int32', element_type=<ElementType....",False,False,756.0,65.0,None,0.0,0.0,0.0,/model-data/aliccp/categories/categories/uniqu...,0,755,item_shop
4,item_brand,"(Tags.ITEM, Tags.CATEGORICAL)","DType(name='int32', element_type=<ElementType....",False,False,756.0,65.0,None,0.0,0.0,0.0,/model-data/aliccp/categories/categories/uniqu...,0,755,item_brand
5,user_shops,"(Tags.CATEGORICAL, Tags.USER)","DType(name='int32', element_type=<ElementType....",False,False,780.0,67.0,None,0.0,0.0,0.0,/model-data/aliccp/categories/categories/uniqu...,0,779,user_shops
6,user_profile,"(Tags.CATEGORICAL, Tags.USER)","DType(name='int32', element_type=<ElementType....",False,False,47.0,16.0,None,0.0,0.0,0.0,/model-data/aliccp/categories/categories/uniqu...,0,46,user_profile
7,user_group,"(Tags.CATEGORICAL, Tags.USER)","DType(name='int32', element_type=<ElementType....",False,False,11.0,16.0,None,0.0,0.0,0.0,/model-data/aliccp/categories/categories/uniqu...,0,10,user_group
8,user_gender,"(Tags.CATEGORICAL, Tags.USER)","DType(name='int32', element_type=<ElementType....",False,False,3.0,16.0,None,0.0,0.0,0.0,/model-data/aliccp/categories/categories/uniqu...,0,2,user_gender
9,user_age,"(Tags.CATEGORICAL, Tags.USER)","DType(name='int32', element_type=<ElementType....",False,False,7.0,16.0,None,0.0,0.0,0.0,/model-data/aliccp/categories/categories/uniqu...,0,6,user_age


#### 3.2 - Designing Retrieval Model Architecture

In [14]:
# Function to create the two tower retrieval model

def create_two_tower(tower_dim: int, encoder_dim: int, optimizer: str, k: int, tags) -> mm.TwoTowerModelV2:
    # User/Query Tower
    user_schema = schema.select_by_tag(tags.USER)
    # create user (query) tower input block
    user_inputs = mm.InputBlockV2(user_schema)
    # create user (query) encoder block
    query = mm.Encoder(
        user_inputs,
        mm.MLPBlock([encoder_dim, tower_dim], no_activation_last_layer=True)
    )

    # Item/Candidate Tower
    item_schema = schema.select_by_tag(tags.ITEM)
    # create item (candidate) tower input block
    item_inputs = mm.InputBlockV2(item_schema)
    # create item (candidate) encoder block
    candidate = mm.Encoder(
        item_inputs,
        mm.MLPBlock([encoder_dim, tower_dim], no_activation_last_layer=True)
    )
    
    # Build Model Class
    model = mm.TwoTowerModelV2(query, candidate)
    model.compile(
        optimizer=optimizer,
        run_eagerly=False,
        loss="categorical_crossentropy",
        metrics=[mm.RecallAt(k), mm.NDCGAt(k)]
    )
    return model

In [15]:
# Initialize model
retrieval_model = create_two_tower(
    tower_dim=64,
    encoder_dim=128,
    optimizer="adam",
    k=10,
    tags=Tags
)

# Fit model
retrieval_model.fit(train_retrieval, validation_data=valid_retrieval, batch_size=4096, epochs=2)

Epoch 1/2
86/86 [==============================] - 121s 1s/step - loss: 8.2951 - recall_at_10: 0.0257 - ndcg_at_10: 0.0243 - regularization_loss: 0.0000e+00 - loss_batch: 8.2778 - val_loss: 8.2924 - val_recall_at_10: 0.0318 - val_ndcg_at_10: 0.0318 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 7.9301
Epoch 2/2
86/86 [==============================] - 148s 2s/step - loss: 8.2946 - recall_at_10: 0.0292 - ndcg_at_10: 0.0290 - regularization_loss: 0.0000e+00 - loss_batch: 8.2772 - val_loss: 8.2928 - val_recall_at_10: 0.0212 - val_ndcg_at_10: 0.0200 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 7.9301


#### 3.3 - Evaluate the model accuracy

In [16]:
# Create candidate/item features for evaluation
candidate_features = unique_rows_by_features(train_retrieval, Tags.ITEM, Tags.ITEM_ID)

# Here's a sneek peek of the item features data
candidate_features.head()

,item_id,item_category,item_shop,item_brand
0,54,54,54,54
1,42,42,42,42
2,6,6,6,6
3,11,11,11,11
4,45,45,45,45


In [17]:
# Convert model to a top_k_encoder
topk_model = retrieval_model.to_top_k_encoder(candidate_features, k=20, batch_size=128)
topk_model.compile(run_eagerly=False)

In [18]:
# Create data loader for validation data
eval_loader = mm.Loader(valid_retrieval, batch_size=1024).map(mm.ToTarget(schema, "item_id"))

# Evaluation
metrics = topk_model.evaluate(eval_loader, return_dict=True)
metrics

147/147 [==============================] - 5s 19ms/step - loss: 0.4305 - recall_at_10: 0.0907 - mrr_at_10: 0.0368 - ndcg_at_10: 0.0494 - map_at_10: 0.0368 - precision_at_10: 0.0091 - regularization_loss: 0.0000e+00 - loss_batch: 0.4310


{'loss': 0.4304984509944916,
 'recall_at_10': 0.09073246270418167,
 'mrr_at_10': 0.0367504358291626,
 'ndcg_at_10': 0.049375735223293304,
 'map_at_10': 0.0367504358291626,
 'precision_at_10': 0.009073246270418167,
 'regularization_loss': 0.0,
 'loss_batch': 0.49215763807296753}

#### 3.4 - Generate top-K candidates

In [19]:
# Create query/user features for evaluation
user_features = unique_rows_by_features(valid_retrieval, Tags.USER, Tags.USER_ID)
user_features.head()

,user_id,user_shops,user_profile,user_group,user_gender,user_age,user_consumption_2,user_is_occupied,user_geography,user_intentions,user_brands,user_categories
0,72,72,3,1,1,1,1,1,1,72,72,72
1,64,64,3,1,1,1,1,1,1,64,64,64
2,12,12,1,1,1,1,1,1,1,12,12,12
4,5,5,1,1,1,1,1,1,1,5,5,5
5,91,91,4,1,1,1,1,1,1,91,91,91


In [20]:
# Check out one batch of 8 Users
loader = mm.Loader(user_features, batch_size=8, shuffle=False)
batch = next(iter(loader))
print(batch[0]['user_id'])

tf.Tensor(
[[72]
 [64]
 [12]
 [ 5]
 [91]
 [35]
 [41]
 [76]], shape=(8, 1), dtype=int32)


In [21]:
scores, reccommended_item_ids = topk_model(batch[0])

In [22]:
# Recommended Items for the batch of 8 Users
reccommended_item_ids

<tf.Tensor: shape=(8, 20), dtype=int32, numpy=
array([[323,  26,  62,   2,   1,  10,  23, 472, 240, 249, 160, 147,  19,
        156, 118, 251,   3, 554, 133, 253],
       [103, 286, 319,  65, 114, 430,   3, 336,  49, 227, 292,  54,  38,
         94, 510, 126, 760,   9,  85,  40],
       [  4,  26,   2,  68,  62, 279, 359, 508,  41, 170, 687, 190, 323,
         58,   8, 472,  23, 410,  87,  10],
       [ 66,   6, 304,  47,  67, 117, 258, 150, 382, 256, 261, 107,  71,
        378,  35, 234,   2,  95,  16,   5],
       [223,  85, 286,   6,  36, 430, 126,  13,  14,   3, 376,  55,   2,
        160, 240, 610,   7, 120, 107, 271],
       [330,  48,   6,  39, 107,   7, 175, 270, 223, 484, 261, 117,  50,
        328,   2, 240, 492, 505, 376, 392],
       [ 15,   4,  20,  13,  42, 159, 437,   1,  44,  29, 121,  83, 361,
        672, 338,   2,  91, 273, 276,  81],
       [ 11,   8,  10, 339,  24,  84,  18, 313, 144,   3, 494, 362, 208,
        223,  32, 269,  38, 195, 490, 232]], dtype=int32)>

In [23]:
# Recommendation "scores" (higher is better) for each recommended item for the batch of 8 Users
scores

<tf.Tensor: shape=(8, 20), dtype=float32, numpy=
array([[0.05276944, 0.05233829, 0.04976109, 0.04800563, 0.04746087,
        0.04671508, 0.04646668, 0.04556989, 0.04400257, 0.04280441,
        0.04248533, 0.04237156, 0.04073053, 0.03999903, 0.03934871,
        0.03901606, 0.03857965, 0.03828777, 0.03811958, 0.03805274],
       [0.130582  , 0.10790169, 0.10362538, 0.09574649, 0.09130216,
        0.09095713, 0.08744904, 0.08681182, 0.08608769, 0.08503264,
        0.08348239, 0.08287771, 0.08158878, 0.08135947, 0.07902861,
        0.07789732, 0.07764154, 0.0769981 , 0.07642979, 0.07535517],
       [0.09923722, 0.09389003, 0.07405806, 0.07362521, 0.07280977,
        0.07133134, 0.0699445 , 0.06551005, 0.0654885 , 0.06486274,
        0.06451014, 0.06421043, 0.06403524, 0.06217494, 0.06024908,
        0.05732848, 0.05724588, 0.05695262, 0.05658302, 0.05632554],
       [0.08351682, 0.07034975, 0.06938601, 0.06599684, 0.06357297,
        0.05978742, 0.05831856, 0.05247527, 0.05225769, 0.050938

### 4 - Ranking Model

#### 4.1 - Feature Engineering for DLRM

In [25]:
train_rank = Dataset(os.path.join(OUTPUT_DATA_DIR, 'train', '*.parquet'), part_size='500MB')
valid_rank = Dataset(os.path.join(OUTPUT_DATA_DIR, 'valid', '*.parquet'), part_size='500MB')

schema = train_rank.schema.without(['user_id_raw', 'item_id_raw'])

In [26]:
schema

,name,tags,dtype,is_list,is_ragged,properties.max_size,properties.start_index,properties.freq_threshold,properties.embedding_sizes.dimension,properties.embedding_sizes.cardinality,properties.cat_path,properties.num_buckets,properties.domain.min,properties.domain.max,properties.domain.name
0,user_id,"(Tags.CATEGORICAL, Tags.USER, Tags.USER_ID, Ta...","DType(name='int32', element_type=<ElementType....",False,False,0.0,0.0,0.0,65.0,747.0,/model-data/aliccp/categories/categories/uniqu...,NaN,0,746,user_id
1,item_id,"(Tags.ITEM, Tags.CATEGORICAL, Tags.ID, Tags.IT...","DType(name='int32', element_type=<ElementType....",False,False,0.0,0.0,0.0,67.0,792.0,/model-data/aliccp/categories/categories/uniqu...,NaN,0,791,item_id
2,item_category,"(Tags.ITEM, Tags.CATEGORICAL)","DType(name='int32', element_type=<ElementType....",False,False,0.0,0.0,0.0,67.0,792.0,/model-data/aliccp/categories/categories/uniqu...,NaN,0,791,item_category
3,item_shop,"(Tags.ITEM, Tags.CATEGORICAL)","DType(name='int32', element_type=<ElementType....",False,False,0.0,0.0,0.0,67.0,792.0,/model-data/aliccp/categories/categories/uniqu...,NaN,0,791,item_shop
4,item_brand,"(Tags.ITEM, Tags.CATEGORICAL)","DType(name='int32', element_type=<ElementType....",False,False,0.0,0.0,0.0,67.0,792.0,/model-data/aliccp/categories/categories/uniqu...,NaN,0,791,item_brand
5,user_shops,"(Tags.CATEGORICAL, Tags.USER)","DType(name='int32', element_type=<ElementType....",False,False,0.0,0.0,0.0,65.0,747.0,/model-data/aliccp/categories/categories/uniqu...,NaN,0,746,user_shops
6,user_profile,"(Tags.CATEGORICAL, Tags.USER)","DType(name='int32', element_type=<ElementType....",False,False,0.0,0.0,0.0,16.0,59.0,/model-data/aliccp/categories/categories/uniqu...,NaN,0,58,user_profile
7,user_group,"(Tags.CATEGORICAL, Tags.USER)","DType(name='int32', element_type=<ElementType....",False,False,0.0,0.0,0.0,16.0,12.0,/model-data/aliccp/categories/categories/uniqu...,NaN,0,11,user_group
8,user_gender,"(Tags.CATEGORICAL, Tags.USER)","DType(name='int32', element_type=<ElementType....",False,False,0.0,0.0,0.0,16.0,3.0,/model-data/aliccp/categories/categories/uniqu...,NaN,0,2,user_gender
9,user_age,"(Tags.CATEGORICAL, Tags.USER)","DType(name='int32', element_type=<ElementType....",False,False,0.0,0.0,0.0,16.0,8.0,/model-data/aliccp/categories/categories/uniqu...,NaN,0,7,user_age


In [27]:
# Target column here is "click" (binary classification)
target_column = schema.select_by_tag(Tags.TARGET).column_names[0]
target_column

'click'

#### 4.2 - Train DLRM

In [30]:
def create_dlrm(optimizer: str, schema, target_column: str, embedding_dim: int):
  model = mm.DLRMModel(
      schema,
      embedding_dim=embedding_dim,
      bottom_block=mm.MLPBlock([128, 64]),
      top_block=mm.MLPBlock([128, 64, 32]),
      prediction_tasks=mm.BinaryClassificationTask(target_column),
  )
  model.compile(optimizer=optimizer, run_eagerly=False, metrics=[tf.keras.metrics.AUC()])
  return model

In [31]:

dlrm = create_dlrm(
    optimizer='adam',
    schema=schema,
    target_column=target_column,
    embedding_dim=64
)

dlrm.fit(train_rank, validation_data=valid_rank, batch_size=16*1024, epochs=3)

Epoch 1/3
43/43 [==============================] - 27s 510ms/step - loss: 0.6932 - auc: 0.4999 - regularization_loss: 0.0000e+00 - loss_batch: 0.6932 - val_loss: 0.6931 - val_auc: 0.5000 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 0.6932
Epoch 2/3
43/43 [==============================] - 21s 469ms/step - loss: 0.6931 - auc: 0.5032 - regularization_loss: 0.0000e+00 - loss_batch: 0.6931 - val_loss: 0.6932 - val_auc: 0.5005 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 0.6933
Epoch 3/3
43/43 [==============================] - 24s 552ms/step - loss: 0.6928 - auc: 0.5141 - regularization_loss: 0.0000e+00 - loss_batch: 0.6928 - val_loss: 0.6936 - val_auc: 0.5007 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 0.6942


In [32]:
# Check out one batch of users
loader = mm.Loader(valid_rank, batch_size=8, shuffle=False)
batch = next(iter(loader))
print(batch[0]['user_id'], batch[0]['item_id'])

tf.Tensor(
[[ 6]
 [72]
 [64]
 [12]
 [73]
 [12]
 [ 4]
 [ 4]], shape=(8, 1), dtype=int32) tf.Tensor(
[[ 11]
 [437]
 [ 26]
 [ 11]
 [  7]
 [ 20]
 [  3]
 [176]], shape=(8, 1), dtype=int32)


In [33]:
# Test the DLRM model I/O with the user batch
dlrm(batch[0])

<tf.Tensor: shape=(8, 1), dtype=float32, numpy=
array([[0.5256107 ],
       [0.49127865],
       [0.48480904],
       [0.4908913 ],
       [0.47849238],
       [0.47689816],
       [0.49065113],
       [0.48722944]], dtype=float32)>

#### 4.3 - Test Retrieval and Rank together

In [34]:
# Test full pipeline
import numpy as np

# Reload Dataset
train_retrieval = Dataset(os.path.join(OUTPUT_RETRIEVAL_DATA_DIR, "train", "*.parquet"), part_size="500MB")
schema = train_retrieval.schema.select_by_tag([Tags.ITEM_ID, Tags.USER_ID, Tags.ITEM, Tags.USER]).without(['click'])
train_retrieval.schema = schema

# User and Item Offline Feature "Stores"
item_fs = unique_rows_by_features(train_retrieval, Tags.ITEM, Tags.ITEM_ID).to_ddf().compute()
user_fs = unique_rows_by_features(train_retrieval, Tags.USER, Tags.USER_ID).to_ddf().compute()

In [35]:
# User batch loader
user_loader = mm.Loader(unique_rows_by_features(train_retrieval, Tags.USER, Tags.USER_ID), batch_size=8, shuffle=False)

# Sample User batch
user_batch = next(iter(user_loader))
users = user_batch[0]['user_id']
users

<tf.Tensor: shape=(8, 1), dtype=int32, numpy=
array([[52],
       [17],
       [ 3],
       [ 2],
       [28],
       [23],
       [ 4],
       [ 6]], dtype=int32)>

In [36]:
# Retrieve candidate Items for each User
_, candidate_item_ids = topk_model(user_batch[0])
candidate_item_ids

<tf.Tensor: shape=(8, 20), dtype=int32, numpy=
array([[ 11,  10,  23, 582, 311,  18, 217,  73, 165, 203, 754, 371,  31,
         78, 399,  26, 152, 223, 147, 177],
       [ 18,   7, 230,  12, 241,  20, 271, 311,   2, 152, 110,  48, 100,
        284,  67, 567,  46, 432,   1,  14],
       [ 18,   3,   2,  67, 415,  12,  48, 383,  14,  20,   7, 433, 223,
          1, 467,  30, 299,   5, 206, 382],
       [258,   6,   9, 250,  65, 413,  66, 280, 306, 373, 234,  15, 107,
         33, 448, 162, 378, 276, 150, 745],
       [  6,   9,   4, 359, 107, 258,  65, 280,  52, 458, 276, 421, 413,
        666, 606, 377,  13, 240, 270,  17],
       [  4,  74,  84, 313, 104,  24,  62, 338,  73, 265,  38, 355, 111,
        170, 208,  53,  44,  99, 174, 232],
       [160, 103,  19, 319,  38, 127, 430, 286, 223, 336, 118,  40, 510,
         49, 126,   3,  84, 227, 232,  85],
       [  2,  10, 284,   4,  26,  15,  12,  83, 128, 595, 106, 359, 582,
         25, 273,  11, 487,  41,  68,  20]], dtype=int32)>

##### 4.4 - Define Functions to run Recommendation Pipeline

In [37]:
# Softmax sample function for ordering
def softmax_sample(recs: np.array, scores: np.array) -> np.array:
    arr = np.exp(scores)/sum(np.exp(scores))
    top_item_idx = (-arr).argsort()
    return recs[top_item_idx]

def generate_recommendations_batch(topk_model, dlrm, input_data, batch_size: int, tags):
    user_loader = mm.Loader(unique_rows_by_features(input_data, tags.USER, tags.USER_ID), batch_size=batch_size, shuffle=False)
    
    # Load a batch
    for batch in user_loader:
            
        # Generate candidates for this batch of users
        users = batch[0]['user_id']
        _, candidate_item_ids = topk_model(batch[0])
        
        # For each user + candidate items, score with the DLRM
        for user, candidates in zip(users.numpy(), candidate_item_ids.numpy()):
            try:
                num_recs = len(candidates)
                user_id = user[0]

                # get user features
                user_features = user_fs[user_fs.user_id == user_id]
                raw_user_id = user_features.user_id_raw.to_numpy()[0]
                user_features = user_features.append([user_features]*(num_recs-1), ignore_index=True)

                # get item features
                item_features = item_fs[item_fs.item_id.isin(candidates)].reset_index(drop=True)
                raw_item_ids = item_features.item_id_raw.to_numpy()

                # combine into feature vectors
                item_features[user_features.columns] = user_features
                item_features = Dataset(item_features)
                item_features.schema = schema.without(['click'])

                # Score with ranking model
                inputs = mm.Loader(item_features, batch_size=num_recs)
                inputs = next(iter(inputs))
                scores = dlrm(inputs[0]).numpy().reshape(-1)

                # Rank
                recs = softmax_sample(raw_item_ids, scores)

                yield raw_user_id, recs
            except Exception as e:
                logging.info(user_id, str(e))

In [38]:
# Test Recommendation Generator
next(generate_recommendations_batch(topk_model, dlrm, train_retrieval, 32, Tags))

(52,
 array([180, 328, 160, 367, 405, 724, 563, 203,  73, 164, 216,  24,  13,
         32,  78, 141, 214,  14,  27,  20], dtype=int32))

### 5.0 - Write Recommendations to Redis

In [39]:
# Import needed Redis libraries
import asyncio
import typing as t
import redis.asyncio as redis
from redis.commands.json.path import Path

In [40]:
async def store_recommendations(recommendations: t.Iterable, redis_conn: redis.Redis):
    """
    Store the recommendations generated for each User in Redis.

    Parameters:
        recommendations (t.Iterable): A generator over a dictionary where the keys are user_ids and the values are lists of recommendations
        redis_conn (redis.Redis): A Redis connection object used to store the recommendations in Redis
    """
    async def store_as_json(user_id: str, recs: list):
        """
        Store an individual user's latest recommendations in Redis.

        Parameters:
            user_id (str): The user id of the user whose recommendations are being stored
            recs (list): A list of item_ids representing the recommendations for the user
        """
        entry = {
            "user_id": int(user_id),
            "recommendations": [int(rec) for rec in recs]
        }
        # Set the JSON object in Redis
        await redis_conn.json().set(f"USER:{user_id}", Path.root_path(), entry)
        
    # Write the recommendations to Redis as a JSON object
    for user_id, recs in recommendations:
        await store_as_json(user_id, recs)

In [ ]:
redis_conn = redis.Redis(
    host="redis-inference-store",
    port=6379,
    decode_responses=True
)

# Create Recommendation Denerator
recommendations = generate_recommendations_batch(topk_model, dlrm, train_retrieval, 32, Tags)

# Run the process - may take a few minutes
await store_recommendations(recommendations, redis_conn=redis_conn)

### 6.0 - Fetch Recommendations from Redis

In [ ]:
!redis-cli -h redis-inference-store -p 6379 JSON.GET USER:1

In [ ]:
# Simple benchmark with async Python client
times = []
for i in range(300):
    t = time.time()
    await redis_conn.json().get(f"USER:{i}")
    times.append(time.time()-t)
    
# Average read time from Redis --> less than a ms likely
np.average(times)

### 7.0 - Export Models

#### 7.1 - Export Retrieval Model

In [42]:
query_tower = retrieval_model.query_encoder
query_tower.save(os.path.join(DATA_DIR, "query_tower"))

## we can load back the saved model via the following script.
#query_tower_loaded = tf.keras.models.load_model(os.path.join(DATA_DIR, 'query_tower'))

#### 7.2 - Export User features

In [43]:
user_features = (
    unique_rows_by_features(train_retrieval, Tags.USER, Tags.USER_ID).compute().reset_index(drop=True)
)
user_features.head()

,user_id,user_shops,user_profile,user_group,user_gender,user_age,user_consumption_2,user_is_occupied,user_geography,user_intentions,user_brands,user_categories,user_id_raw
0,52,52,2,1,1,1,1,1,1,52,52,52,52
1,17,17,1,1,1,1,1,1,1,17,17,17,19
2,3,3,1,1,1,1,1,1,1,3,3,3,6
3,2,2,1,1,1,1,1,1,1,2,2,2,7
4,28,28,2,1,1,1,1,1,1,28,28,28,29


In [44]:
# save to disk
user_features.to_parquet(os.path.join(DATA_DIR, "user_features.parquet"))

#### 7.3 - Export User Embeddings

In [45]:
queries = retrieval_model.query_embeddings(Dataset(user_features, schema=schema), batch_size=1024, index=Tags.USER_ID)
query_embs_df = queries.compute(scheduler="synchronous").reset_index()
query_embs_df.head()

,user_id,0,1,2,3,4,5,6,7,8,...,54,55,56,57,58,59,60,61,62,63
0,1,-0.016454,0.047637,0.029430,0.092070,-0.075310,-0.117258,0.081995,-0.023881,0.049352,...,0.011308,-0.038586,0.066274,-0.018696,-0.004772,-0.067845,-0.024231,-0.041651,-0.034216,0.041892
1,2,-0.039006,0.044807,0.068528,0.093152,-0.034810,-0.134904,0.115000,0.035052,0.026317,...,0.106941,-0.072939,-0.000363,-0.059570,-0.016762,-0.073508,-0.005920,-0.022326,0.042812,-0.047774
2,3,0.011051,0.014181,-0.037957,0.043957,-0.002348,-0.042717,0.015754,-0.042029,0.000412,...,0.012987,-0.003837,-0.037139,-0.063552,0.002379,-0.042614,0.053593,-0.020704,-0.000197,-0.062537
3,4,-0.106151,-0.060785,0.000231,0.109255,-0.012758,-0.019461,0.015303,0.042368,0.036709,...,-0.135305,-0.057978,0.020739,-0.088541,-0.021950,-0.010913,-0.002898,-0.066684,0.050496,0.023917
4,5,0.037376,0.159237,-0.034339,0.105674,0.028274,-0.136207,0.179024,0.006649,0.004559,...,0.158877,-0.037146,0.020592,-0.021437,0.015181,-0.115882,0.059757,0.037434,-0.067330,-0.017049


#### 7.4 - Export Item Features

In [46]:
item_features = (
    unique_rows_by_features(train_retrieval, Tags.ITEM, Tags.ITEM_ID).compute().reset_index(drop=True)
)
item_features.head()

,item_id,item_category,item_shop,item_brand,item_id_raw
0,54,54,54,54,54
1,42,42,42,42,42
2,6,6,6,6,10
3,11,11,11,11,14
4,45,45,45,45,45


In [48]:
# save to disk
item_features.to_parquet(os.path.join(DATA_DIR, "item_features.parquet"))

#### 7.5 - Export Item Embeddings

In [49]:
item_embs = retrieval_model.candidate_embeddings(Dataset(item_features, schema=schema), batch_size=1024, index=Tags.ITEM_ID)
item_embs_df = item_embs.compute(scheduler="synchronous")
item_embs_df.head()

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
item_id,,,,,,,,,,,,,,,,,,,,,
1,0.009732,-0.014411,-0.025075,0.012731,0.042892,0.024582,0.029094,0.003211,0.057809,0.005961,...,0.015897,-0.012351,-0.050376,0.016657,-0.057117,-0.037514,-0.000722,0.042838,-0.101512,-0.016187
2,0.010512,-0.017656,-0.036645,0.038234,-0.023894,-0.009890,-0.002719,-0.026165,-0.006086,0.045234,...,0.021852,0.039206,0.016535,-0.029534,-0.077412,-0.102906,-0.028267,0.003728,-0.009214,0.050523
3,-0.015020,-0.040423,-0.023804,0.083483,0.036492,0.038706,0.008003,0.031845,-0.003107,-0.011783,...,0.000122,0.029737,-0.048085,0.020916,-0.067554,-0.025193,-0.024575,0.016857,-0.003245,0.043760
4,-0.014155,0.005807,0.062907,-0.029907,-0.074642,-0.075029,-0.002133,-0.052205,0.049959,-0.049147,...,0.051226,-0.042476,0.020111,-0.006088,-0.072868,0.006863,-0.019055,-0.032266,-0.111518,0.074673
5,0.007159,-0.036682,-0.030354,0.040060,0.022868,-0.043924,0.000604,0.006278,0.051364,0.008314,...,0.045578,0.017455,-0.022009,0.003558,-0.030128,-0.020089,-0.030706,0.023854,-0.046757,0.037810


In [50]:
# save to disk
item_embs_df.to_parquet(os.path.join(DATA_DIR, "item_embeddings.parquet"))

7.6 - Export DLRM

In [51]:
dlrm.save(os.path.join(DATA_DIR, "dlrm"))

In [55]:
!zip -r model /model-data

  adding: model-data/ (stored 0%)
  adding: model-data/aliccp/ (stored 0%)
  adding: model-data/aliccp/dlrm/ (stored 0%)
  adding: model-data/aliccp/dlrm/.merlin/ (stored 0%)
  adding: model-data/aliccp/dlrm/.merlin/input_schema.json (deflated 89%)
  adding: model-data/aliccp/dlrm/.merlin/output_schema.json (deflated 24%)
  adding: model-data/aliccp/dlrm/variables/ (stored 0%)
  adding: model-data/aliccp/dlrm/variables/variables.index (deflated 69%)
  adding: model-data/aliccp/dlrm/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: model-data/aliccp/dlrm/keras_metadata.pb (deflated 94%)
  adding: model-data/aliccp/dlrm/assets/ (stored 0%)
  adding: model-data/aliccp/dlrm/fingerprint.pb (stored 0%)
  adding: model-data/aliccp/dlrm/saved_model.pb (deflated 91%)
  adding: model-data/aliccp/user_features.parquet (deflated 77%)
  adding: model-data/aliccp/processed/ (stored 0%)
  adding: model-data/aliccp/processed/train/ (stored 0%)
  adding: model-data/aliccp/processed/train/